# Runtime
Calls the ML models, handles training, loads in the dataset, using all the other methods present in ./Experiments and ./Models

## Methods and Libraries

In [5]:
import os
import sys
import torch
import wandb
import logging
import pickle
from torch.utils.data import DataLoader

%run -i config.ipynb
%run -i train.ipynb

%run -i ../Models/net_conv.ipynb
%run -i ../Models/net_conv_rf.ipynb
%run -i ../Models/net_rnn.ipynb

%run -i ../Models/utils.ipynb

## Argument and Configuration Handler 

In [2]:
# Determine MODEL to use
MODEL = 'null'
models = ['CDIL', 'DIL', 'TCN', 'CNN', 'Deformable', 'LSTM', 'GRU']
while MODEL not in models:
    print('Select one of the following models to train on the dataset: ', end='')
    for i in models:
        print(i, ' ', end='')
    MODEL = input('Selection:')


# Configure device (CPU)
device = 'cpu'

# Using WANDB for visualization of validation and performance
wandb.init(project='DeFungi', name=MODEL, entity="jacob-davidian")
WANDB = wandb

# Config parameters
cfg_training = config['training']
cfg_model = config['models']

BATCH = cfg_training['batch_size']
CLASS = cfg_model['n_class']
SEQ_LEN = cfg_model['n_length']
FIX_length = cfg_model['fix_length']
USE_EMBED = cfg_model['use_embedding']
CHAR_COCAB = cfg_model['vocab_size']
INPUT_SIZE = cfg_model['dim']

Select one of the following models to train on the dataset: CDIL  DIL  TCN  CNN  Deformable  LSTM  GRU  

Selection: CDIL


wandb: Currently logged in as: hawkhobo (jacob-davidian). Use `wandb login --relogin` to force relogin


## Model Selection

In [3]:
# Model selection with configs 
if MODEL == 'CDIL' or MODEL == 'DIL' or MODEL == 'TCN' or MODEL == 'CNN':
    LAYER = cfg_model['cnn_layer']
    NHID = cfg_model['cnn_hidden']
    KERNEL_SIZE = cfg_model['cnn_ks']
    net = CONV(MODEL, INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, False, False, USE_EMBED, CHAR_COCAB, FIX_length)
    receptive_field(seq_length=SEQ_LEN, model=MODEL, kernel_size=KERNEL_SIZE, layer=LAYER)
elif MODEL == 'Deformable':
    LAYER = cfg_model['cnn_layer']
    NHID = cfg_model['cnn_hidden']
    KERNEL_SIZE = cfg_model['cnn_ks']
    net = CONV('CNN', INPUT_SIZE, CLASS, [NHID] * LAYER, KERNEL_SIZE, True, False, USE_EMBED, CHAR_COCAB, FIX_length)
    receptive_field(seq_length=SEQ_LEN, model=MODEL, kernel_size=KERNEL_SIZE, layer=LAYER)
elif MODEL == 'LSTM' or MODEL == 'GRU':
    LAYER = cfg_model['rnn_layer']
    NHID = cfg_model['rnn_hidden']
    net = RNN(MODEL, INPUT_SIZE, CLASS, NHID, LAYER, USE_EMBED, CHAR_COCAB, FIX_length)

net = net.to(device)
para_num = sum(p.numel() for p in net.parameters() if p.requires_grad)

# Prinout CNN architecture
print('Diagram of CNN: \n', net)

position: 512
layer:1 	 distance:3 	 size:3
layer:2 	 distance:7 	 size:7
layer:3 	 distance:15 	 size:15
layer:4 	 distance:31 	 size:31
layer:5 	 distance:63 	 size:63
layer:6 	 distance:127 	 size:127
layer:7 	 distance:255 	 size:255
layer:8 	 distance:511 	 size:511
layer:9 	 distance:1023 	 size:1023

Diagram of CNN: 
 CONV(
  (embedding): Embedding(256, 64)
  (conv): ConvPart(
    (conv_net): Sequential(
      (0): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (1): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (2): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,), padding_mode=circular)
        (nonlinear): ReLU()
      )
      (3): Block(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), di

/home/hawkobo/anaconda3/envs/jupyterlab-debugger/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## Create Log files
Please consult the readme for what each file does

In [4]:
file_name = 'P' + str(para_num) + '_' + MODEL + '_L' + str(LAYER) + '_H' + str(NHID)

os.makedirs('DeFungi_log', exist_ok=True)
os.makedirs('DeFungi_model', exist_ok=True)
log_file_name = './DeFungi_log/' + file_name + '.txt'
model_name = './DeFungi_model/' + file_name + '.ph'
handlers = [logging.FileHandler(log_file_name), logging.StreamHandler()]
logging.basicConfig(level=logging.INFO, format='%(message)s', handlers=handlers)
loginf = logging.info

loginf(device)
loginf(file_name)

cpu
P128453_CDIL_L9_H64


## Create Optimizer, Load Data (Train/Val/Test)

In [13]:
# Optimize
optimizer = torch.optim.Adam(net.parameters())
loss = torch.nn.CrossEntropyLoss(reduction='sum')


# Data
trainloader = DataLoader(DeFungiDataset(f'../defungi_encoded/image.train.pickle', True), batch_size=BATCH, shuffle=True, drop_last=False)
valloader = DataLoader(DeFungiDataset(f'../defungi_encoded/image.dev.pickle', True), batch_size=BATCH, shuffle=False, drop_last=False)
testloader = DataLoader(DeFungiDataset(f'../defungi_encoded/image.test.pickle', False), batch_size=BATCH, shuffle=False, drop_last=False)

Loaded ../defungi_encoded/image.train.pickle... size=8
Loaded ../defungi_encoded/image.dev.pickle... size=1
Loaded ../defungi_encoded/image.test.pickle... size=1


## Perform Training

In [16]:
TrainModel(
    fix_length=FIX_length,
    net=net,
    device=device,
    trainloader=trainloader,
    valloader=valloader,
    testloader=testloader,
    n_epochs=cfg_training['epoch'],
    n_classes=cfg_model['n_class'],
    optimizer=optimizer,
    loss=loss,
    loginf=loginf,
    wandb=WANDB,
    file_name=model_name
)


  0%|                                                     | 0/1 [00:00<?, ?it/s]


TypeError: new(): invalid data type 'bytes'

## Pass to Front-End

In [ ]:
# Save Training Results for front-end
filename = 'fungi_classifier.sav'
pickle.dump(f'DeFungi_{MODEL}', open(filename, 'wb'))